### Flight Cleaning Notebook
This notebook will be responsible for merging all the flight information for the San Diego International Airport's On Time performance covering two years.

In [114]:
# import libraries
import pandas as pd

In [115]:
# import the csv files
Jan23 = pd.read_csv('Raw_data/raw_on-time/Jan23_OT_report.csv')
Feb23 = pd.read_csv('Raw_data/raw_on-time/Feb23_OT_report.csv')
Mar23 = pd.read_csv('Raw_data/raw_on-time/Mar23_OT_report.csv')
Apr23 = pd.read_csv('Raw_data/raw_on-time/Apr23_OT_report.csv')
May23 = pd.read_csv('Raw_data/raw_on-time/May23_OT_report.csv')
Jun23 = pd.read_csv('Raw_data/raw_on-time/Jun23_OT_report.csv')
Jul23 = pd.read_csv('Raw_data/raw_on-time/Jul23_OT_report.csv')
Aug23 = pd.read_csv('Raw_data/raw_on-time/Aug23_OT_report.csv')
Sep23 = pd.read_csv('Raw_data/raw_on-time/Sep23_OT_report.csv')
Oct23 = pd.read_csv('Raw_data/raw_on-time/Oct23_OT_report.csv')
Nov23 = pd.read_csv('Raw_data/raw_on-time/Nov23_OT_report.csv')
Dec23 = pd.read_csv('Raw_data/raw_on-time/Dec23_OT_report.csv')

In [116]:
# list the df's in chronological order
dfs = [Jan23, Feb23, Mar23, Apr23, May23, Jun23, Jul23, Aug23, Sep23, Oct23, Nov23, Dec23]

In [117]:
# merge df's
merged_df = pd.concat(dfs, ignore_index=True)

In [118]:
# Filter all df so that only flights leaving San Diego are represented
SAN_df = merged_df.loc[merged_df['ORIGIN'] == 'SAN']
# Reset the index
SAN_df = SAN_df.reset_index(drop=True)

In [119]:
# Get shape of the df
SAN_df.shape

(90955, 17)

In [120]:
# get the data types of the df
SAN_df.dtypes

FL_DATE                 object
OP_UNIQUE_CARRIER       object
OP_CARRIER_FL_NUM        int64
ORIGIN                  object
DEST                    object
CRS_DEP_TIME             int64
DEP_TIME               float64
DEP_DEL15              float64
DEP_DELAY_GROUP        float64
DEP_TIME_BLK            object
CANCELLED              float64
CANCELLATION_CODE       object
CARRIER_DELAY          float64
WEATHER_DELAY          float64
NAS_DELAY              float64
SECURITY_DELAY         float64
LATE_AIRCRAFT_DELAY    float64
dtype: object

In [121]:
print(SAN_df.head())

                FL_DATE OP_UNIQUE_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST  \
0  1/1/2023 12:00:00 AM                AA               1055    SAN  DFW   
1  1/1/2023 12:00:00 AM                AA               1651    SAN  CLT   
2  1/1/2023 12:00:00 AM                AA               1663    SAN  ORD   
3  1/1/2023 12:00:00 AM                AA               1765    SAN  DFW   
4  1/1/2023 12:00:00 AM                AA               1939    SAN  DFW   

   CRS_DEP_TIME  DEP_TIME  DEP_DEL15  DEP_DELAY_GROUP DEP_TIME_BLK  CANCELLED  \
0           715     713.0        0.0             -1.0    0700-0759        0.0   
1           615     644.0        1.0              1.0    0600-0659        0.0   
2          2259    2250.0        0.0             -1.0    2200-2259        0.0   
3           815     810.0        0.0             -1.0    0800-0859        0.0   
4          1450    1447.0        0.0             -1.0    1400-1459        0.0   

  CANCELLATION_CODE  CARRIER_DELAY  WEATHER_DELAY  NAS_D

In [122]:
# adjust the FL_DATE column to datetime
SAN_df['FL_DATE'] = pd.to_datetime(SAN_df['FL_DATE']).dt.date

C:\Users\graha\AppData\Local\Temp\ipykernel_41204\1251424886.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  SAN_df['FL_DATE'] = pd.to_datetime(SAN_df['FL_DATE']).dt.date


In [123]:
# convert CRS_DEP_TIME to a string
SAN_df['CRS_DEP_TIME'] = SAN_df['CRS_DEP_TIME'].astype(str).str.zfill(4)
# convert CRS_DEP_TIME to time format
SAN_df['CRS_DEP_TIME'] = pd.to_datetime(SAN_df['CRS_DEP_TIME'], format='%H%M').dt.time

In [124]:
print(SAN_df.head())

      FL_DATE OP_UNIQUE_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST CRS_DEP_TIME  \
0  2023-01-01                AA               1055    SAN  DFW     07:15:00   
1  2023-01-01                AA               1651    SAN  CLT     06:15:00   
2  2023-01-01                AA               1663    SAN  ORD     22:59:00   
3  2023-01-01                AA               1765    SAN  DFW     08:15:00   
4  2023-01-01                AA               1939    SAN  DFW     14:50:00   

   DEP_TIME  DEP_DEL15  DEP_DELAY_GROUP DEP_TIME_BLK  CANCELLED  \
0     713.0        0.0             -1.0    0700-0759        0.0   
1     644.0        1.0              1.0    0600-0659        0.0   
2    2250.0        0.0             -1.0    2200-2259        0.0   
3     810.0        0.0             -1.0    0800-0859        0.0   
4    1447.0        0.0             -1.0    1400-1459        0.0   

  CANCELLATION_CODE  CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  SECURITY_DELAY  \
0               NaN            NaN        

In [125]:
# create function to handle missing DEP_TIME for cancelled flights
def convert_dep_time(val):
    if pd.isnull(val):
        return None
    try:
        val_float = float(val)
        val_int = int(val_float)
        time_str = str(val_int).zfill(4)
        return pd.to_datetime(time_str, format='%H%M').time()
    except Exception:
        return None

In [126]:
# apply the function to the DEP_TIME column
SAN_df['DEP_TIME'] = SAN_df['DEP_TIME'].apply(convert_dep_time)

In [127]:
print(SAN_df.head())

      FL_DATE OP_UNIQUE_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST CRS_DEP_TIME  \
0  2023-01-01                AA               1055    SAN  DFW     07:15:00   
1  2023-01-01                AA               1651    SAN  CLT     06:15:00   
2  2023-01-01                AA               1663    SAN  ORD     22:59:00   
3  2023-01-01                AA               1765    SAN  DFW     08:15:00   
4  2023-01-01                AA               1939    SAN  DFW     14:50:00   

   DEP_TIME  DEP_DEL15  DEP_DELAY_GROUP DEP_TIME_BLK  CANCELLED  \
0  07:13:00        0.0             -1.0    0700-0759        0.0   
1  06:44:00        1.0              1.0    0600-0659        0.0   
2  22:50:00        0.0             -1.0    2200-2259        0.0   
3  08:10:00        0.0             -1.0    0800-0859        0.0   
4  14:47:00        0.0             -1.0    1400-1459        0.0   

  CANCELLATION_CODE  CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  SECURITY_DELAY  \
0               NaN            NaN        

In [128]:
# convert the DEP_DEL15 column to int
SAN_df['DEP_DEL15'] = SAN_df['DEP_DEL15'].fillna(0).astype(int)

In [129]:
# convert CANCELLED column to int
SAN_df['CANCELLED'] = SAN_df['CANCELLED'].fillna(0).astype(int)

In [130]:
# check what values are present in cancellation_code
SAN_df['CANCELLATION_CODE'].value_counts()

CANCELLATION_CODE
B    435
A    354
C    117
D      1
Name: count, dtype: int64

In [131]:
# fill missing values with None in CANCELLATION_CODE
SAN_df['CANCELLATION_CODE'] = SAN_df['CANCELLATION_CODE'].fillna('None')

In [132]:
# check what values are present in CARRIER_DELAY
SAN_df['CARRIER_DELAY'].value_counts()


CARRIER_DELAY
0.0      10114
1.0        421
2.0        418
3.0        360
4.0        356
         ...  
674.0        1
126.0        1
216.0        1
207.0        1
659.0        1
Name: count, Length: 389, dtype: int64

In [133]:
# convert the CARRIER_DELAY column to int
SAN_df['CARRIER_DELAY'] = SAN_df['CARRIER_DELAY'].fillna(0).astype(int)

In [134]:
# convert the WEATHER_DELAY column to int
SAN_df['WEATHER_DELAY'] = SAN_df['WEATHER_DELAY'].fillna(0).astype(int)

In [135]:
# convert the NAS_DELAY column to int
SAN_df['NAS_DELAY'] = SAN_df['NAS_DELAY'].fillna(0).astype(int)

In [136]:
# convert the SECURITY_DELAY column to int
SAN_df['SECURITY_DELAY'] = SAN_df['SECURITY_DELAY'].fillna(0).astype(int)

In [137]:
# convert the LATE_AIRCRAFT_DELAY column to int
SAN_df['LATE_AIRCRAFT_DELAY'] = SAN_df['LATE_AIRCRAFT_DELAY'].fillna(0).astype(int)

In [139]:
# reorder the columns so times are next to the date
SAN_df = SAN_df[['FL_DATE', 'CRS_DEP_TIME', 'DEP_TIME', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'DEP_DEL15', 'CANCELLED', 'CANCELLATION_CODE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']]
print(SAN_df.head())

      FL_DATE CRS_DEP_TIME  DEP_TIME OP_UNIQUE_CARRIER  OP_CARRIER_FL_NUM  \
0  2023-01-01     07:15:00  07:13:00                AA               1055   
1  2023-01-01     06:15:00  06:44:00                AA               1651   
2  2023-01-01     22:59:00  22:50:00                AA               1663   
3  2023-01-01     08:15:00  08:10:00                AA               1765   
4  2023-01-01     14:50:00  14:47:00                AA               1939   

  ORIGIN DEST  DEP_DEL15  CANCELLED CANCELLATION_CODE  CARRIER_DELAY  \
0    SAN  DFW          0          0              None              0   
1    SAN  CLT          1          0              None              0   
2    SAN  ORD          0          0              None              0   
3    SAN  DFW          0          0              None              0   
4    SAN  DFW          0          0              None              0   

   WEATHER_DELAY  NAS_DELAY  SECURITY_DELAY  LATE_AIRCRAFT_DELAY  
0              0          0          

In [140]:
# Save the df to a csv file
SAN_df.to_csv('clean_data/SAN_OT_report.csv', index=False)